<a href="https://colab.research.google.com/github/Rishita77/PythonMajorProject/blob/main/Uniqlo_Major_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics

! pip install chart_studio

from chart_studio.plotly import iplot
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')
%pylab inline

!pwd

from google.colab import files

uploaded = files.upload()
datatrain = pd.read_csv("/content/UniqloTrainingstocks1216.csv")

datatrain.head()

chart1 = go.Figure(data=[go.Candlestick(x=datatrain['Date'],
                open=datatrain['Open'],
                high=datatrain['High'],
                low=datatrain['Low'],
                close=datatrain['Close'])])

chart1.show()

#EDA
features = ['Open', 'High', 'Low', 'Close', 'Volume']

plt.subplots(figsize=(20,10))

for i, col in enumerate(features):
  plt.subplot(2,3,i+1)
  sns.distplot(datatrain[col])
plt.show()

# From the distribution plots we can see that the Volume data is left skewed and across OHLC, there are two major peaks

plt.subplots(figsize=(20,10))
for i, col in enumerate(features):
  plt.subplot(2,3,i+1)
  sns.boxplot(datatrain[col])
plt.show()

#From the boxplots we can see that the Volume data has outliers

splitted = datatrain['Date'].str.split('-', expand=True)

datatrain['Year'] = splitted[0].astype('int')
datatrain['Month'] = splitted[1].astype('int')
datatrain['Day'] = splitted[2].astype('int')

datatrain['is_quarter_end'] = np.where(datatrain['Month']%3==0,1,0)

data_grouped = datatrain.groupby('Year').mean()
plt.subplots(figsize=(20,10))

for i, col in enumerate(['Open', 'High', 'Low', 'Close']):
  plt.subplot(2,2,i+1)
  data_grouped[col].plot.bar()
plt.show()

#We can see from the quarterly data that Uniqlo's stock price increases by almost 2.5x from 2012 - 2015 and then reduced in 2016

datatrain.groupby('is_quarter_end').mean()

#Prices and volumes traded are higher at the end of a quarter

#Modelling
datatrain['open-close']  = datatrain['Open'] - datatrain['Close']
datatrain['low-high']  = datatrain['Low'] - datatrain['High']
datatrain['target'] = np.where(datatrain['Close'].shift(-1) > datatrain['Close'], 1, 0)

features = datatrain[['open-close', 'low-high', 'is_quarter_end']]
target = datatrain['target']

scaler = StandardScaler()
features = scaler.fit_transform(features)

X_train, X_test, Y_train, Y_test = train_test_split(
    features, target, test_size=0.1, random_state=2022)
print(X_train.shape, X_test.shape)

models = [LogisticRegression(), SVC(
  kernel='poly', probability=True), XGBClassifier()]

for i in range(3):
  models[i].fit(X_train, Y_train)

  print(f'{models[i]} : ')
  print('Training Accuracy : ', metrics.roc_auc_score(
    Y_train, models[i].predict_proba(X_train)[:,1]))
  print('Validation Accuracy : ', metrics.roc_auc_score(
    Y_test, models[i].predict_proba(X_test)[:,1]))
  print()

#While XBGClassifier has the highest accuracy for training data, the large difference between the accuracy for training and test data shows it's prone to overfitting for which reason Logistic Regression is the best method to be used here

Populating the interactive namespace from numpy and matplotlib
/content


In [ ]:
# Code snippet for ensemble learning
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
voting_classifier = VotingClassifier(estimators=[('lr', LogisticRegression()), ('svc', SVC()), ('xgb', XGBClassifier())], voting='hard')
voting_classifier.fit(X_train, Y_train)

voting_predictions = voting_classifier.predict(X_test)

accuracy_voting = accuracy_score(Y_test, voting_predictions)
precision_voting = precision_score(Y_test, voting_predictions)
recall_voting = recall_score(Y_test, voting_predictions)
f1_voting = f1_score(Y_test, voting_predictions)

print(f"Ensemble Model (Voting)")
print(f"Accuracy: {accuracy_voting:.4f}")
print(f"Precision: {precision_voting:.4f}")
print(f"Recall: {recall_voting:.4f}")
print(f"F1 Score: {f1_voting:.4f}")


Ensemble Model (Voting)
Accuracy: 0.7561
Precision: 0.7333
Recall: 0.7586
F1 Score: 0.7458


In [ ]:
# Import necessary libraries
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Ensure scikit-learn is up-to-date
!pip install --upgrade scikit-learn

# Create a StackingClassifier with Logistic Regression as the meta-classifier
stacking_classifier = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression()),
        ('svc', SVC()),
        ('xgb', XGBClassifier())
    ],
    final_estimator=LogisticRegression()
)

# Fit the stacking classifier on the training data
stacking_classifier.fit(X_train, Y_train)

# Make predictions on the test data
stacking_predictions = stacking_classifier.predict(X_test)

# Evaluate the performance of the stacking classifier
accuracy_stacking = accuracy_score(Y_test, stacking_predictions)
precision_stacking = precision_score(Y_test, stacking_predictions)
recall_stacking = recall_score(Y_test, stacking_predictions)
f1_stacking = f1_score(Y_test, stacking_predictions)

# Print the performance metrics
print("Ensemble Model (Stacking)")
print(f"Accuracy: {accuracy_stacking:.4f}")
print(f"Precision: {precision_stacking:.4f}")
print(f"Recall: {recall_stacking:.4f}")
print(f"F1 Score: {f1_stacking:.4f}")


Ensemble Model (Stacking)
Accuracy: 0.7561
Precision: 0.7333
Recall: 0.7586
F1 Score: 0.7458


In [ ]:
# Assuming you have a trained XGBoost model 'xgb_model'
booster = XGB.get_booster()
feature_importances = booster.get_fscore()

# Now 'feature_importances' is a dictionary where keys are feature names and values are their importance scores
